In [1]:
cd ../

C:\Users\yulya\PycharmProjects\machine_learning_of_patterns


In [2]:
import time
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
from random import randint, random
from typing import Callable, Any, Tuple, List, Optional
from tqdm import tqdm

In [3]:
from regex.generator import RegexGenerator
from regex.parser import RegexParser

In [7]:
def generate_stateful_regex(
        length: int,
        star_num: int, 
        star_nesting: int, 
        alphabet_size: int = 26) -> str:
    gen = RegexGenerator(length, star_num, star_nesting, alphabet_size)
    regex = ""
    while len(regex) != length:
        regex = gen.generate_regex()
    return regex



def generate_word(
        length: int, 
        alphabet_size: int = 26,
        alphabet: Optional[List[str]] = None) -> str:
    if alphabet is None:
        alphabet = list(string.ascii_lowercase)[:alphabet_size] + \
                   list(string.ascii_uppercase)[:alphabet_size]
    word = ''.join([alphabet[randint(0, len(alphabet) - 1)] for _ in range(length)])
    return word
    


def measure_time(func: Callable, *params) -> Tuple: # in seconds
    t1 = time.perf_counter()
    result = func(*params)
    t2 = time.perf_counter()
    return result, t2 - t1


def count_items_in_regex(regex: str, alphabet_size: int = 26) -> Tuple[int]:
    stars = 0
    letters = 0
    alphabet = list(string.ascii_lowercase)[:alphabet_size] + \
               list(string.ascii_uppercase)[:alphabet_size]
    for c in regex:
        if c in alphabet:
            letters += 1
        elif c == '*':
            stars += 1
    return letters, stars


def get_letters(word: str) -> List[str]:
    letters = set()
    alphabet = list(string.ascii_lowercase)[:alphabet_size] + \
               list(string.ascii_uppercase)[:alphabet_size]
    for c in word:
        if c in alphabet:
            letters.add(c)
    return list(letters)

In [8]:
regex_length = [i for i in range(5, 31, 5)]
alphabet_size = 26

In [9]:
results = {
    "regex": [],
    "length": [],
    "word": [],
    "matching_type": [],
    "time": [],
    "match": []
}

for length in regex_length:
    star_nesting = star_num = length // 2
    regex_str = generate_stateful_regex(length, star_num, star_nesting)
    letters, stars = count_items_in_regex(regex_str)
    word = generate_word(letters ** stars, alphabet=get_letters(regex_str))
    
    parser = RegexParser(regex_str, [100] * star_num)
    regex = parser.parse()
    nfa = regex.to_nfa()
    dfa = nfa.to_dfa()
    
    data = {
        "regex": regex,
        "nfa": nfa,
        "dfa": dfa
    }
    for k, v in data.items():
        match, t = measure_time(v.match, word)
        results["regex"].append(regex_str)
        results["length"].append(len(regex_str))
        results["word"].append(word)
        results["matching_type"].append(k)
        results["time"].append(t)
        results["match"].append(match)

In [10]:
df = pd.DataFrame(data=results)

In [11]:
df

,regex,length,word,matching_type,time,match
0,P*|L*,5,LLLP,regex,0.000031,False
1,P*|L*,5,LLLP,nfa,0.000030,False
2,P*|L*,5,LLLP,dfa,0.000004,False
3,pY*(U*)O*i,10,ppOiUOYUiYOYpOYYipYpiYpUpYiUipiUpUiYiYYOOpYYpY...,regex,0.000066,False
4,pY*(U*)O*i,10,ppOiUOYUiYOYpOYYipYpiYpUpYiUipiUpUiYiYYOOpYYpY...,nfa,0.000036,False
5,pY*(U*)O*i,10,ppOiUOYUiYOYpOYYipYpiYpUpYiUipiUpUiYiYYOOpYYpY...,dfa,0.000010,False
6,(z*|V|(b*))*(a),15,VVzbzbVazVbzbVVbVVaaazzVabzaazzabzVazaaVazVzbV...,regex,0.000077,False
7,(z*|V|(b*))*(a),15,VVzbzbVazVbzbVVbVVaaazzVabzaazzabzVazaaVazVzbV...,nfa,0.000113,False
8,(z*|V|(b*))*(a),15,VVzbzbVazVbzbVVbVVaaazzVabzaazzabzVazaaVazVzbV...,dfa,0.000006,False
9,(E|A)i|(h*g*QlN(R))*,20,QQiQgREiiNiiiRNNQlANiEAhlARAhhAARNRNlQNNihRREQ...,regex,0.000020,False


In [12]:
df.to_csv('./experiments/matching_time.csv', index=False)